<a href="https://colab.research.google.com/github/jordanburdett/ensembleTesting/blob/master/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn as sk

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostClassifier

Datasets

In [ ]:
heartData = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv", na_values='?')
forestFire = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv", na_values='?')
clientSubscribe = pd.read_csv("https://raw.githubusercontent.com/jordanburdett/ensembleTesting/master/bank-full.csv", sep=';')

In [ ]:
def makeCatCodes(data):
  for label,dtype in data.dtypes.items():
    if dtype == object:
        
        # set the dataframe to be a category
        data[label] = data[label].astype('category')

        # create new row using cat codes
        data["{}_cat".format(label)] = data[label].cat.codes
        
  return data

In [ ]:
heartData = makeCatCodes(heartData)
forestFire = makeCatCodes(forestFire)
clientSubscribe = makeCatCodes(clientSubscribe)

HeartData
  This data is used to predict the death of those that had some sort of heart disease. Using many different attributes to predict. This I will be using a neural network

In [ ]:
features = heartData[["age","anaemia","creatinine_phosphokinase","diabetes","ejection_fraction","high_blood_pressure","platelets","serum_creatinine","serum_sodium","sex","smoking","time"]].to_numpy()
targets = heartData["DEATH_EVENT"].to_numpy() # 1 is death 0 is alive

features = normalize(features)
train_data, test_data, train_targets, test_targets = train_test_split(features, targets, test_size=.3)

In [ ]:
model = Sequential()
model.add(Dense(250, activation='relu', input_dim=12))
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_targets, epochs=1000)

Epoch 1/1000
209/209 [==============================] - 0s 459us/step - loss: 0.6308 - accuracy: 0.7081
Epoch 2/1000
209/209 [==============================] - 0s 93us/step - loss: 0.6065 - accuracy: 0.7081
Epoch 3/1000
209/209 [==============================] - 0s 77us/step - loss: 0.6057 - accuracy: 0.7081
Epoch 4/1000
209/209 [==============================] - 0s 72us/step - loss: 0.6045 - accuracy: 0.7081
Epoch 5/1000
209/209 [==============================] - 0s 71us/step - loss: 0.6040 - accuracy: 0.7081
Epoch 6/1000
209/209 [==============================] - 0s 75us/step - loss: 0.6054 - accuracy: 0.7081
Epoch 7/1000
209/209 [==============================] - 0s 66us/step - loss: 0.6044 - accuracy: 0.7081
Epoch 8/1000
209/209 [==============================] - 0s 65us/step - loss: 0.6058 - accuracy: 0.7081
Epoch 9/1000
209/209 [==============================] - 0s 68us/step - loss: 0.6039 - accuracy: 0.7081
Epoch 10/1000
209/209 [==============================] - 0s 67us/step - 

In [ ]:
scores = model.evaluate(train_data, train_targets, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test = model.predict(test_data)
scores2 = model.evaluate(test_data, test_targets, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

Accuracy on training data: 0.8468899726867676% 
 Error on training data: 0.15311002731323242
Accuracy on test data: 0.7888888716697693% 
 Error on test data: 0.2111111283302307


In [ ]:
model = GradientBoostingClassifier(n_estimators=1000, learning_rate=.1, max_depth=1, random_state=1)
model.fit(train_data, train_targets)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
model.score(test_data, test_targets)

0.8333333333333334

In [ ]:
model = AdaBoostClassifier(n_estimators=1000)
model.fit(train_data, train_targets)
model.score(test_data, test_targets)

0.7888888888888889

Forest fire area prediction

In [ ]:
forestFire.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,month_cat,day_cat
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,7,0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10,5
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10,2
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,7,0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,7,3


In [ ]:
features = forestFire[["X",	"Y", "FFMC",	"DMC",	"DC",	"ISI",	"temp",	"RH",	"wind",	"rain", "month_cat",	"day_cat"]].to_numpy()
#features = normalize(features)

targets = forestFire["area"].to_numpy()

train_data, test_data, train_targets, test_targets = train_test_split(features, targets, test_size=.3)

In [ ]:
from sklearn import svm
model = svm.SVR(kernel="rbf")
model.fit(train_data, train_targets)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
predictions = model.predict(test_data)

In [ ]:
import sklearn.metrics as metrics

# variance score
print("Variance Score")
print(metrics.explained_variance_score(test_targets, predictions))

# max error
from sklearn.metrics import max_error
print("max error")
print(max_error(test_targets, predictions))

# mean absolute error
from sklearn.metrics import mean_absolute_error
print("mean absolute error")
print(mean_absolute_error(test_targets, predictions))

# Mean squared error
from sklearn.metrics import mean_squared_error
print("mean squared error")
print(mean_squared_error(test_targets, predictions))

# r2 score
print ("r2 score")
print (metrics.r2_score(test_targets, predictions))

model.score(test_data, test_targets)

Variance Score
0.0003088804949138435
max error
1089.8666953368463
mean absolute error
16.487076367159816
mean squared error
8589.857339966786
r2 score
-0.029262424693105027


-0.029262424693105027

In [ ]:
model = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01)

In [ ]:
model.fit(train_data, train_targets)
model.score(test_data, test_targets)

-0.10818279304266178

In [ ]:
predictions = model.predict(test_data)
import sklearn.metrics as metrics

# variance score
print("Variance Score")
print(metrics.explained_variance_score(test_targets, predictions))

# max error
from sklearn.metrics import max_error
print("max error")
print(max_error(test_targets, predictions))

# mean absolute error
from sklearn.metrics import mean_absolute_error
print("mean absolute error")
print(mean_absolute_error(test_targets, predictions))

# Mean squared error
from sklearn.metrics import mean_squared_error
print("mean squared error")
print(mean_squared_error(test_targets, predictions))


# r2 score
print ("r2 score")
print (metrics.r2_score(test_targets, predictions))

Variance Score
-0.10811287356285337
max error
985.9236040538196
mean absolute error
28.204243355336423
mean squared error
9248.49860489245
r2 score
-0.10818279304266176


In [ ]:
forestFire.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,month_cat,day_cat
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,7,0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10,5
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10,2
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,7,0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,7,3


clientSubscribe data

In [ ]:
clientSubscribe.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_cat,marital_cat,education_cat,default_cat,housing_cat,loan_cat,contact_cat,month_cat,poutcome_cat,y_cat
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,4,1,2,0,1,0,2,8,3,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,9,2,1,0,1,0,2,8,3,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,2,1,1,0,1,1,2,8,3,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,1,1,3,0,1,0,2,8,3,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,11,2,3,0,0,0,2,8,3,0


In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [ ]:
features = clientSubscribe[["balance", "day", "duration", "pdays", "previous", "job_cat",	"marital_cat",	"education_cat",	"default_cat",	"housing_cat",	"loan_cat",	"contact_cat",	"month_cat",	"poutcome_cat"]].to_numpy()
#features = normalize(features)

targets = clientSubscribe["y"].to_numpy()

train_data, test_data, train_targets, test_targets = train_test_split(features, targets, test_size=.3)

In [ ]:
model = KNN(n_neighbors=6)
model.fit(train_data, train_targets)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [ ]:
model.score(test_data, test_targets)

0.8839575346505456

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier()
model.fit(train_data, train_targets)

HistGradientBoostingClassifier(l2_regularization=0.0, learning_rate=0.1,
                               loss='auto', max_bins=255, max_depth=None,
                               max_iter=100, max_leaf_nodes=31,
                               min_samples_leaf=20, n_iter_no_change=None,
                               random_state=None, scoring=None, tol=1e-07,
                               validation_fraction=0.1, verbose=0,
                               warm_start=False)

In [ ]:
model.score(test_data, test_targets)

0.9084340902388676